# Holder Wallet Conformance Flows (v3) - CTWalletCrossDeferred

# 0.0 Initial setup

## 0.1 Setup conformance

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [2]:
from eudi_wallet.did_key import KeyDid, PublicKeyJWK
import uuid

# generate crypto seed
crypto_seed = b'ebsitests'

key_did = KeyDid(seed=crypto_seed)

# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9KbptQwoaj2VP1S6Ahzo7REFCT4NBTPYdQinCZbCcyoqWKi9Q2uEW36DNSXhCwiYnGz6BAZkzytQAEBE5cPidCGnadH4SsLDbSZeG2SEChrqvQpdK4Mk8H32vs3B5g8Wr7kcc


## 1.1 Initiate Credential Issuance

In [3]:
from eudi_wallet.util import parse_query_string_parameters_from_url
from eudi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type =  CredentialTypes.CTWalletCrossDeferred
# qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
qr_code_data = "openid-credential-offer://?credential_offer_uri=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fissuer-mock%2Foffers%2Fa349a26b-a624-4df6-b722-33b1f5662a72"
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)


[18:47:38] Credential offer URI:                                                                   ]8;id=628159;file:///tmp/ipykernel_72801/2441860016.py\2441860016.py]8;;\:]8;id=534075;file:///tmp/ipykernel_72801/2441860016.py#16\16]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/a349a26b-a624-4df6-b7                 
           22-33b1f5662a72                                                                                         

{'credential_issuer': 'https://api-conformance.ebsi.eu/conformance/v3/issuer-mock', 'credentials': [{'format': 'jwt_vc', 'types': ['VerifiableCredential', 'VerifiableAttestation', 'CTWalletCrossDeferred'], 'trust_framework': {'name': 'ebsi', 'type': 'Accreditation', 'uri': 'TIR link towards accreditation'}}], 'grants': {'authorization_code': {'issuer_state': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW95dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpOVEydUVXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRLNE1rOEgzMnZzM0I1ZzhXcjdrY2MiLCJjcmVkZW50aWFsX3R5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl0sImlhdCI6MTY4ODQ3NjYxNywiZXhwIjoxNjg4NDc2OTE3LCJpc3MiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIiwiYXV

[18:47:39] Credential offer:                                                                       ]8;id=370387;file:///tmp/ipykernel_72801/2441860016.py\2441860016.py]8;;\:]8;id=810358;file:///tmp/ipykernel_72801/2441860016.py#19\19]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletCrossDeferred'                                                                 
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'authorization_code': {                                                                         
                       'issuer_state':                                                                             
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNF                 
           ZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1                 
           BnWW95dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpO                 
           VEydUVXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRL                 
           NE1rOEgzMnZzM0I1ZzhXcjdrY2MiLCJjcmVkZW50aWFsX3R5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                 
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl0sImlhdCI6MTY4ODQ3NjYxNy                 
           wiZXhwIjoxNjg4NDc2OTE3LCJpc3MiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hb                 
           mNlL3YzL2lzc3Vlci1tb2NrIiwiYXVkIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25mb3Jt                 
           YW5jZS92My9hdXRoLW1vY2siLCJzdWIiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW9                 
           5dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpOVEydU                 
           VXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRLNE1rO                 
           EgzMnZzM0I1ZzhXcjdrY2MifQ.94X6Fwth0LUuU4yRrK1yNdCYsmzU3Cn7WKPn9xa2kTCwCBEevb5JopaazZzCb                 
           sM0FHafW0yGqMS94_FTpRwiHg'    

           Credential issuer configuration:                                                        ]8;id=600936;file:///tmp/ipykernel_72801/2441860016.py\2441860016.py]8;;\:]8;id=333149;file:///tmp/ipykernel_72801/2441860016.py#22\22]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cre                 
           dential',                                                                                               
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer                 
           -mock/credential_deferred',                                                                             
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework':

[18:47:40] Authorization server configuration:                                                     ]8;id=898638;file:///tmp/ipykernel_72801/2441860016.py\2441860016.py]8;;\:]8;id=729760;file:///tmp/ipykernel_72801/2441860016.py#25\25]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/au                 
           thorize',                                                                                               
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Perform authorization request and obtain ID token request

In [4]:
from eudi_wallet.siop_auth.util import (
    perform_authorization, 
    AuthorizationRequestQueryParams,
    get_authorization_response_query_params,
    generate_code_challenge,
    generate_code_verifier
)
import uuid
import json

state = str(uuid.uuid4())
nonce = str(uuid.uuid4())
code_verifier = generate_code_verifier()

authorization_details = [{"type":"openid_credential","format":"jwt_vc","types":credential_offer.credentials[0].get("types"),"locations": ["https://api-conformance.ebsi.eu/conformance/v3/issuer-mock"]}]
redirect_uri = "http://localhost:8080"
client_metadata = {"vp_formats_supported":{"jwt_vp":{"alg":["ES256"]},"jwt_vc":{"alg":["ES256"]}},"response_types_supported":["vp_token","id_token"],"authorization_endpoint":redirect_uri}
authorization_request_query_params = AuthorizationRequestQueryParams(
    response_type="code",
    scope="openid",
    state=state,
    client_id=key_did._did,
    authorization_details=json.dumps(authorization_details, separators=(',', ':')),
    redirect_uri=redirect_uri,
    nonce=nonce,
    code_challenge=generate_code_challenge(code_verifier),
    code_challenge_method="S256",
    client_metadata=json.dumps(client_metadata, separators=(',', ':')),
    issuer_state=credential_offer.grants.get("authorization_code").get("issuer_state")
)

auth_resp = await perform_authorization(auth_server_configuration.authorization_endpoint, authorization_request_query_params)
auth_resp_uri = str(auth_resp).split("Location': '")[1].split("'")[0]
console.log("Authorization response URI: ", auth_resp_uri)
auth_resp_query_params = get_authorization_response_query_params(auth_resp_uri)
console.log("Authorization response query params: ", auth_resp_query_params)

[18:47:46] Authorization response URI:                                                              ]8;id=364196;file:///tmp/ipykernel_72801/554801659.py\554801659.py]8;;\:]8;id=695553;file:///tmp/ipykernel_72801/554801659.py#34\34]8;;\
           http://localhost:8080?state=1261839f-5ed4-429d-85ac-7336c5b168d4&client_id=https%3A%2F%2                
           Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock&redirect_uri=https%3A%2F%2Fapi-c                
           onformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock%2Fdirect_post&response_type=id_token&r                
           esponse_mode=direct_post&scope=openid&nonce=9f07eb8a-3534-4cee-978e-3c3f8dbbffd7&request                
           _uri=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock%2Frequest_uri%                
           2F321a3ed1-fa43-4d03-bc33-d142309682b3                                                                  

           Authorization response query params:                                                     ]8;id=410510;file:///tmp/ipykernel_72801/554801659.py\554801659.py]8;;\:]8;id=356535;file:///tmp/ipykernel_72801/554801659.py#36\36]8;;\
           AuthorizationResponseQueryParams(                                                                       
               state='1261839f-5ed4-429d-85ac-7336c5b168d4',                                                       
               client_id='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                               
               redirect_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post',                
               response_type='id_token',                                                                           
               response_mode='direct_post',                                                                        
               scope='openid',                                                                                     
               nonce='9f07eb8a-3534-4cee-978e-3c3f8dbbffd7',                                                       
               request_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/request_uri/32                
           1a3ed1-fa43-4d03-bc33-d142309682b3',                                                                    
               presentation_definition='',                                                                         
               request=''                                                                                          
           )                                                                                                       

## 1.3 Send ID token

In [5]:
from eudi_wallet.siop_auth.util import send_id_token_response

id_token = key_did.generate_id_token(auth_server_uri=auth_resp_query_params.client_id, nonce=auth_resp_query_params.nonce)
auth_code_response = await send_id_token_response(auth_resp_query_params.redirect_uri, id_token, auth_resp_query_params.state)
auth_code_response = str(auth_code_response).split("Location': '")[1].split("'")[0]
state = parse_query_string_parameters_from_url(auth_code_response).get("state")[0]
auth_code = parse_query_string_parameters_from_url(auth_code_response).get("code")[0]
console.log("Authorization code: ", auth_code)

[18:47:49] Authorization code:  e3da2d52-3dfc-4fa0-aacc-51aa3b13bf4a                                ]8;id=584380;file:///tmp/ipykernel_72801/3477489019.py\3477489019.py]8;;\:]8;id=343963;file:///tmp/ipykernel_72801/3477489019.py#8\8]8;;\

## 1.4 Exchange code for access token

In [6]:
from eudi_wallet.siop_auth.util import exchange_auth_code_for_access_token

token_uri = auth_resp_query_params.client_id + "/token"
access_token_response = await exchange_auth_code_for_access_token(token_uri, key_did._did, auth_code, code_verifier)
console.log("Access token response: ", access_token_response)

[18:47:52] Access token response:                                                                    ]8;id=998984;file:///tmp/ipykernel_72801/376351808.py\376351808.py]8;;\:]8;id=925106;file:///tmp/ipykernel_72801/376351808.py#5\5]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWV               
           ZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjRiOTUxY2NmLTBiNTMtNDhjOC1iNGU5LWZiY2VhY               
           jg2ODlmZiIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudG               
           lhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV               
           1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVy               
           aWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl19XSwiY19ub25jZSI6Ijc5OTE1N2Q3L               
           TE0NTItNDg2OC05MjI5LTQ2NDJmY2Q5NGE0YiIsImNfbm9uY2VfZXhwaXJlc19pbiI6ODY0MDAsImNsaWVudF9pZC               
           I6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJ               
           WUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFF               
           QkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyJ9LCJpc3MiOiJod               
           HRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2F1dGgtbW9jayIsImF1ZCI6WyJodH               
           RwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInN1YiI6ImR               
           pZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFT               
           NkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y               
           1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsImlhdCI6MTY4ODQ3Nj               
           Y3MSwiZXhwIjoxNjg4NTYzMDcxfQ.FehqUd8UtEqeqwFFcLBbZVfVRNVoTSRCMG15GTqTe418bXjWJJUXATzizNKa               
           DAACH98OVN-EIJe8YhMKlKwOPw',                                                                            
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWl               
           GbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjY4ZTg4MTIyLTM5MmItNGRhYS1hODZlLWUxMDE2ZjhlM               
           jI1YiIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2               
           NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J               
           wdFF3b2FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJB               
           Wmt6eXRRQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwiY               
           XVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2               
           FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXR               
           RQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwiaWF0Ijox               
           Njg4NDc2NjcxLCJleHAiOjE2ODg0NzY3OTF9.fpW_iBwt9GksRQwQCMsQOMtRHlNSyy1S2D6VdKXJoH-MXuEVwJ7d               
           ts25tQdZsKP1VU07Jr6rb44xBw3bvvduCQ',                                                                    
               c_nonce='799157d7-1452-4868-9229-4642fcd94a4b',                                                     
               c_nonce_expires_in=86400     

## 1.5 Request credential to get acceptance token to initiate deferred flow 

In [7]:
from eudi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
acceptance_token_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
console.log("Acceptance token response: ", acceptance_token_response)## 1.6 Request credential using deferred endpoint

[18:47:54] Credential request JWT:                                                                  ]8;id=695913;file:///tmp/ipykernel_72801/3783255632.py\3783255632.py]8;;\:]8;id=252076;file:///tmp/ipykernel_72801/3783255632.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtV                
           WjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkRO                
           U1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMz                
           QjVnOFdyN2tjYyN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2Fq                
           MlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXRR                
           QUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwidHlwIjoi                
           b3BlbmlkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2Nvb                
           mZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjg4NTYzMDc0LCJpYXQiOjE2ODg0NzY2NzQsImlzcyI6I                
           mRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWU                
           DFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQ                
           kU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsIm5vbmNlIjoiN                
           zk5MTU3ZDctMTQ1Mi00ODY4LTkyMjktNDY0MmZjZDk0YTRiIn0.mNzEr4Uhh-zWGUDh7kwssIPdULEJ4-410FuBz                
           C62xzR1VmAjfUqk_7ZtN75x6OxmVDcG9dr052jSviBmClA3FQ                                                       

           Acceptance token response:                                                               ]8;id=980109;file:///tmp/ipykernel_72801/3783255632.py\3783255632.py]8;;\:]8;id=575614;file:///tmp/ipykernel_72801/3783255632.py#6\6]8;;\
           {'acceptance_token': 'TwjFl8ZHkvhZfWuIg_uWfqbnilXBQZDNA-n4Td2SyiU'}                                     

## 1.6 Request credential using deferred endpoint

In [8]:
from eudi_wallet.siop_auth.util import send_deferred_credential_request
from time import sleep

console.log("Waiting for 5 seconds...")
sleep(5)
console.log("Sending deferred credential request...")
credential_response = await send_deferred_credential_request(credential_issuer_configuration.deferred_credential_endpoint, acceptance_token_response.get("acceptance_token"))
print("Credential response: ", credential_response)

[18:47:57] Waiting for 5 seconds...                                                                 ]8;id=258464;file:///tmp/ipykernel_72801/4037029182.py\4037029182.py]8;;\:]8;id=702066;file:///tmp/ipykernel_72801/4037029182.py#4\4]8;;\

[18:48:02] Sending deferred credential request...                                                   ]8;id=810909;file:///tmp/ipykernel_72801/4037029182.py\4037029182.py]8;;\:]8;id=531561;file:///tmp/ipykernel_72801/4037029182.py#6\6]8;;\

Credential response:  {'format': 'jwt_vc', 'credential': 'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNvbmZvcm1hbmNlIzE3OWY1NmFmLWI3NzYtNDBlMS04N2EyLTczYTJjN2JkZjkyZCIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsImlzcyI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjg4NDc2Njc0LCJleHAiOjE2ODg1NjMwNzQsImlhdCI6MTY4ODQ3NjY3NCwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwiaWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlIzE3OWY1NmFmLWI3NzYtNDBlMS04N2EyLTczYTJjN2JkZjkyZCIsInR5cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldENyb3NzRGVmZXJyZWQiXSwiaXNzdWVyIjoiZGlkOmVic2k6emhKQVJqUE42OWNFdGdQeEh